# Script for a period

In [1]:
import requests
import json
import pandas as pd
from clickhouse_connect import get_client
from datetime import datetime, timedelta, date
from dotenv import load_dotenv
import os
import time

load_dotenv()

True

In [2]:
KeyGuten = os.getenv('KeyGuten')
KeyGiper = os.getenv('KeyGiper')
KeyKitchen = os.getenv('KeyKitchen')
KeySmart = os.getenv("KeySmart")

## Списки кампаний

Link: https://dev.wildberries.ru/openapi/promotion#tag/Kampanii/paths/~1adv~1v1~1promotion~1count/get

In [3]:


# API endpoint
url = 'https://advert-api.wildberries.ru/adv/v1/promotion/count'

# Headers including the API key for authentication
headers_guten = {
    'Authorization': KeyGuten,
    'Accept': 'application/json'
}

headers_giper = {
    'Authorization': KeyGiper,
    'Accept': 'application/json'
}

headers_kitchen = {
    'Authorization': KeyKitchen,
    'Accept': 'application/json'
}

headers_smart = {
    'Authorization': KeySmart,
    'Accept': 'application/json'
}

# Make the GET request
response_guten = requests.get(url, headers=headers_guten)
response_giper = requests.get(url, headers=headers_giper)
response_kitchen = requests.get(url, headers=headers_kitchen)
response_smart = requests.get(url, headers=headers_smart)

# Check if the request was successful
if response_guten.status_code == 200 and response_giper.status_code == 200 and response_kitchen.status_code == 200 and response_smart.status_code == 200:
    # Parse the JSON response
    data_guten = response_guten.json()
    data_giper = response_giper.json()
    data_kitchen = response_kitchen.json()
    data_smart = response_smart.json()
    print("Data retrieved successfully")
else:
    print(f"Failed to retrieve data. Status code: {response_guten.status_code,response_giper.status_code,response_giper.status_code,response_smart.status_code }")
    print(f"Response: {response_guten.text, response_giper.text,response_giper.text,response_smart.text }")

Data retrieved successfully


In [4]:
def process_advert_data(data):
    # Flatten the JSON data
    df = pd.json_normalize(
        data['adverts'],
        record_path='advert_list', 
        meta=['type', 'status', 'count']
    )
    
    # Convert 'changeTime' to datetime format
    df['changeTime'] = pd.to_datetime(df['changeTime'])
    
    # Reset the index
    df = df.reset_index(drop=True)
    
    return df

# Now you can process each dataset with one line:
df_guten = process_advert_data(data_guten)
df_giper = process_advert_data(data_giper)
df_kitchen = process_advert_data(data_kitchen)
df_smart = process_advert_data(data_smart)

### Информация о кампаниях


Link: https://dev.wildberries.ru/openapi/promotion#tag/Kampanii/paths/~1adv~1v1~1promotion~1adverts/post

In [5]:
# Define the API endpoint
url = "https://advert-api.wildberries.ru/adv/v1/promotion/adverts"

# Define the query parameters
query_params = {
    "order": "create",  # Order by the "change" field
    "direction":"desc"
}


# Function to fetch campaign data
def fetch_campaign_data(df, headers, project_name, chunk_size=50):
    chunks = [df['advertId'][i:i + chunk_size].tolist() for i in range(0, len(df), chunk_size)]
    all_campaign_data = []
    for idx, chunk in enumerate(chunks):
        response = requests.post(url, params=query_params, json=chunk, headers=headers)
        if response.status_code == 200:
            data = response.json()
            all_campaign_data.extend(data)
            print(f"Data retrieved successfully for {project_name} chunk {idx + 1}")
        else:
            print(f"Error for {project_name} chunk {idx + 1}: {response.status_code}, {response.text}")
        time.sleep(1)  # Add a delay to avoid hitting API rate limits
    
    all_campaign_data = pd.DataFrame(all_campaign_data)
    all_campaign_data = all_campaign_data.sort_values(by='createTime', ascending=False)
    all_campaign_data['Project'] = project_name
    all_campaign_data['Marketplace'] = 'Wildberries'
    
    return all_campaign_data

# Fetch campaign data for each project
campaigns_guten = fetch_campaign_data(df_guten, headers_guten, 'WB-GutenTech')
campaigns_giper = fetch_campaign_data(df_giper, headers_giper, 'WB-ГиперМаркет')
campaigns_kitchen = fetch_campaign_data(df_kitchen, headers_kitchen, 'WB-KitchenAid')
campaigns_smart = fetch_campaign_data(df_smart, headers_smart, 'WB-Smart-Market')

# Concatenate the DataFrames
combined_campaigns = pd.concat([campaigns_guten, campaigns_giper, campaigns_kitchen,campaigns_smart], ignore_index=True)

print("Combined Campaign Data")
#campaigns_kitchen

Data retrieved successfully for WB-GutenTech chunk 1
Data retrieved successfully for WB-GutenTech chunk 2
Data retrieved successfully for WB-GutenTech chunk 3
Data retrieved successfully for WB-GutenTech chunk 4
Data retrieved successfully for WB-GutenTech chunk 5
Data retrieved successfully for WB-GutenTech chunk 6
Data retrieved successfully for WB-GutenTech chunk 7
Data retrieved successfully for WB-GutenTech chunk 8
Data retrieved successfully for WB-GutenTech chunk 9
Data retrieved successfully for WB-ГиперМаркет chunk 1
Data retrieved successfully for WB-ГиперМаркет chunk 2
Data retrieved successfully for WB-KitchenAid chunk 1
Data retrieved successfully for WB-Smart-Market chunk 1
Combined Campaign Data


In [6]:
# Keep only the desired columns
columns_to_keep = ["endTime", "createTime", "startTime","name", "advertId", "status", "type","Project","Marketplace"]
# Ensure filtered_df is a copy of the slice, not a view
filtered_df = combined_campaigns[columns_to_keep].copy()
filtered_df['endTime'] = pd.to_datetime(filtered_df['endTime'], format='mixed').dt.date
filtered_df['createTime'] = pd.to_datetime(filtered_df['createTime'], format='mixed').dt.date
filtered_df['startTime'] = pd.to_datetime(filtered_df['startTime'], format='mixed').dt.date

# Mapping dictionaries for 'status' and 'type'
status_mapping = {
    -1: "Кампания в процессе удаления",
    4: "Готова к запуску",
    7: "Кампания завершена",
    8: "Отказался",
    9: "Идут показы",
    11: "Кампания на паузе"
}

type_mapping = {
    4: "Кампания в каталоге (устаревший тип)",
    5: "Кампания в карточке товара (устаревший тип)",
    6: "Кампания в поиске (устаревший тип)",
    7: "Кампания в рекомендациях на главной странице (устаревший тип)",
    8: "Автоматическая кампания",
    9: "Аукцион"
}

# Replace numeric values with their string descriptions
filtered_df['status'] = filtered_df['status'].replace(status_mapping)
filtered_df['type'] = filtered_df['type'].replace(type_mapping)

# Display the updated DataFrame
filtered_df

,endTime,createTime,startTime,name,advertId,status,type,Project,Marketplace
0,2100-01-01,2025-05-14,2025-05-14,SMEG_Чайники для плиты_СЛИВ_14.05.2025_3%,25517545,Идут показы,Автоматическая кампания,WB-GutenTech,Wildberries
1,2100-01-01,2025-05-14,2025-05-14,SMEG_Чайники электро_СЛИВ_14.05.2025_3%,25517467,Идут показы,Автоматическая кампания,WB-GutenTech,Wildberries
2,2100-01-01,2025-05-14,2025-05-14,SMEG_Тостеры_СЛИВ_14.05.2025_3%,25517139,Идут показы,Автоматическая кампания,WB-GutenTech,Wildberries
3,2100-01-01,2025-05-14,2025-05-14,SMEG_Соковыжималки_СЛИВ_14.05.2025_3%,25517053,Идут показы,Автоматическая кампания,WB-GutenTech,Wildberries
4,2100-01-01,2025-05-14,2025-05-14,SMEG_Миксеры_СЛИВ_14.05.2025_3%,25517025,Идут показы,Автоматическая кампания,WB-GutenTech,Wildberries
...,...,...,...,...,...,...,...,...,...
516,2024-06-23,2024-03-21,2024-04-18,Насадки Oral-B Автореклама,15562382,Кампания завершена,Автоматическая кампания,WB-Smart-Market,Wildberries
517,2100-01-01,2024-03-21,2025-02-20,Braun_Электробритвы_22.10.2024,15562363,Идут показы,Автоматическая кампания,WB-Smart-Market,Wildberries
518,2024-10-11,2024-03-21,2024-07-02,Электрические зубные щетки Oral-B Поиск + Каталог,15562321,Кампания завершена,Аукцион,WB-Smart-Market,Wildberries
519,2024-10-15,2024-03-21,2024-08-27,Эпиляторы Braun_7 SKU_27.08.2024,15562282,Кампания завершена,Автоматическая кампания,WB-Smart-Market,Wildberries


## Статистика кампаний


Link: https://dev.wildberries.ru/openapi/analytics#tag/Statistika-po-prodvizheniyu/paths/~1adv~1v2~1fullstats/post

Данные вернутся для кампаний в статусах:

* 7 — завершена
* 9 — идут показы
* 11 — пауза из-за расхода бюджета

Response Schema: 

* dates	/ Array of strings <date> [ items <date > ]/ Даты, за которые необходимо выдать информацию.
* views	/ integer/ Количество просмотров./ За все дни, по всем артикулам WB и платформам.
* clicks	/ integer/ Количество кликов./ За все дни, по всем артикулам WB и платформам.
* ctr	/ number/ Показатель кликабельности./ Отношение числа кликов к количеству показов. Выражается в процентах./ За все дни, по всем артикулам WB и платформам.
* cpc	/ number/ Средняя стоимость клика, ₽./ За все дни, по всем артикулам WB и платформам.
* sum	/ number/ Затраты, ₽./ За все дни, по всем артикулам WB и платформам.
* atbs	/ integer/ Количество добавлений товаров в корзину./ За все дни, по всем артикулам WB и платформам.
* orders	/ integer/ Количество заказов./ За все дни, по всем артикулам WB и платформам.
* cr	/ integer/ CR(conversion rate) — это отношение количества заказов к общему количеству посещений кампании./ За все дни, по всем артикулам WB и платформам.
* shks	/ integer/ Количество заказанных товаров, шт./ За все дни, по всем артикулам WB и платформам.
* sum_price	/ number/ Заказов на сумму, ₽/ За все дни, по всем артикулам WB и платформам.
* days	/ Array of objects (Days)/ Статистка по дням
    Array 
    * date	/ string <date-time>/ Дата, за которую представлены данные/ 
    * views	/ integer/ Количество просмотров/ 
    * clicks	/ integer/ Количество кликов/ 
    * ctr	/ number/ Показатель кликабельности, отношение числа кликов к количеству показов, %/ 
    * cpc	/ number/ Средняя стоимость клика, ₽./ 
    * sum	/ number/ Затраты, ₽./ 
    * atbs	/ integer/ Количество добавлений товаров в корзину/ 
    * orders	/ integer/ Количество заказов/ 
    * cr	/ integer/ CR(conversion rate) — отношение количества заказов к общему количеству посещений кампании/ 
    * shks	/ integer/ Количество заказанных товаров, шт./ 
    * sum_price	/ number/ Заказов на сумму, ₽/ 
    * apps	/ Array of objects/ Блок информации о платформе
    Array 
        - views	/ integer/ Количество просмотров/ 
        - clicks	/ integer/ Количество кликов/ 
        - ctr	/ number/ Показатель кликабельности, отношение числа кликов к количеству показов, %/ 
        - cpc	/ number/ Средняя стоимость клика, ₽./ 
        - sum	/ number/ Затраты, ₽./ 
        - atbs	/ integer/ Количество добавлений товаров в корзину/ 
        - orders	/ integer/ Количество заказов/ 
        - cr	/ integer/ CR(conversion rate) — это отношение количества заказов к общему количеству посещений кампании/ 
        - shks	/ integer/ Количество заказанных товаров, шт./ 
        - sum_price	/ number/ Заказов на сумму, ₽/ 
        - nm	/ Array of objects/ Блок статистики по артикулам WB/ 
        - appType	/ integer/ Тип платформы (1 - сайт, 32 - Android, 64 - IOS)
* boosterStats	/ Array of objects (BoosterStats)/ Статистика по средней позиции товара на страницах поисковой выдачи и каталога (для автоматических кампаний).
* advertId	/ integer/ ID кампании

In [7]:
# API endpoint
url = "https://advert-api.wildberries.ru/adv/v2/fullstats"
# Automatically get yesterday's date
yesterday = "2025-05-14"
specific_date = str(yesterday)  # Replace with your desired date

### Guten

In [8]:
def fetch_campaign_stats(campaign_ids, headers, date, base_url,project_name, chunk_size=100):
    chunks = [campaign_ids['advertId'][i:i + chunk_size].tolist() for i in range(0, len(campaign_ids), chunk_size)]
    all_data = []
    for idx, chunk in enumerate(chunks): 
        #Construct the payload for the current chunk
        payload = [{"id": campaign_id, "dates": [date]} for campaign_id in chunk]

        # Send the POST request
        response = requests.post(base_url, headers=headers, json=payload)

        # Check the response status
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            all_data.extend(data)
            print(f"Data retrieved successfully for chunk {idx + 1} for {project_name}" )
        else:
            print(f"Error for chunk {idx + 1}: {response.status_code}, {response.text}")
            if "no companies with correct intervals" in response.text:
                print("Stopping execution due to invalid interval error")
                break
            
        # Add a delay to avoid hitting API rate limits
        time.sleep(65)
            
    return all_data if all_data else [{}] 

# Fetch campaign data
all_campaign_data_guten = fetch_campaign_stats(campaigns_guten, headers_guten, specific_date, url, "WB-GutenTech")
all_campaign_data_giper = fetch_campaign_stats(campaigns_giper, headers_giper, specific_date, url, "WB-ГиперМаркет")
all_campaign_data_kitchen = fetch_campaign_stats(campaigns_kitchen, headers_kitchen, specific_date, url, "WB-KitchenAid")
all_campaign_data_smart = fetch_campaign_stats(campaigns_smart, headers_smart, specific_date, url, "WB-Smart-Market")


Data retrieved successfully for chunk 1 for WB-GutenTech
Data retrieved successfully for chunk 2 for WB-GutenTech
Error for chunk 3: 400, {"error":"Invalid Params: there are no companies with correct intervals"}
Stopping execution due to invalid interval error
Data retrieved successfully for chunk 1 for WB-ГиперМаркет
Data retrieved successfully for chunk 1 for WB-KitchenAid
Data retrieved successfully for chunk 1 for WB-Smart-Market


In [9]:
all_campaign_data_giper

[{'views': 1886,
  'clicks': 53,
  'ctr': 2.81,
  'cpc': 21.55,
  'sum': 1141.91,
  'atbs': 6,
  'orders': 1,
  'cr': 1.89,
  'shks': 1,
  'sum_price': 3615,
  'dates': ['2025-05-14'],
  'days': [{'date': '2025-05-14T03:00:00+03:00',
    'views': 1886,
    'clicks': 53,
    'ctr': 0,
    'cpc': 21.55,
    'sum': 1141.91,
    'atbs': 6,
    'orders': 1,
    'cr': 1.89,
    'shks': 1,
    'sum_price': 3615,
    'apps': [{'views': 332,
      'clicks': 9,
      'ctr': 2.71,
      'cpc': 20.68,
      'sum': 186.11,
      'atbs': 0,
      'orders': 0,
      'cr': 0,
      'shks': 0,
      'sum_price': 0,
      'nm': [{'views': 112,
        'clicks': 1,
        'ctr': 0.89,
        'cpc': 59.97,
        'sum': 59.97,
        'atbs': 0,
        'orders': 0,
        'cr': 0,
        'shks': 0,
        'sum_price': 0,
        'name': 'Тонометр автоматический на плечо UA-888AC адаптер, чехол',
        'nmId': 236036418},
       {'views': 109,
        'clicks': 3,
        'ctr': 2.75,
        'cpc

In [10]:
all_campaign_data_smart

[{'views': 1351,
  'clicks': 74,
  'ctr': 5.48,
  'cpc': 1.83,
  'sum': 135.14,
  'atbs': 18,
  'orders': 7,
  'cr': 9.46,
  'shks': 7,
  'sum_price': 101961,
  'dates': ['2025-05-14'],
  'days': [{'date': '2025-05-14T03:00:00+03:00',
    'views': 1351,
    'clicks': 74,
    'ctr': 0.01,
    'cpc': 1.83,
    'sum': 135.14,
    'atbs': 18,
    'orders': 7,
    'cr': 9.46,
    'shks': 7,
    'sum_price': 101961,
    'apps': [{'views': 83,
      'clicks': 4,
      'ctr': 4.82,
      'cpc': 2.46,
      'sum': 9.85,
      'atbs': 1,
      'orders': 1,
      'cr': 25,
      'shks': 1,
      'sum_price': 20256,
      'nm': [{'views': 10,
        'clicks': 1,
        'ctr': 10,
        'cpc': 1,
        'sum': 1,
        'atbs': 0,
        'orders': 0,
        'cr': 0,
        'shks': 0,
        'sum_price': 0,
        'name': 'Электрическая зубная щетка iO 7 White Alabaster',
        'nmId': 206314538},
       {'views': 11,
        'clicks': 0,
        'ctr': 0,
        'cpc': 0,
        'sum

In [11]:
def flatten_campaign_data(campaign_data):
    flattened_data = []
    
    for entry in campaign_data:
        advertId = entry.get("advertId")
        for day in entry.get("days", []):
            date = day.get("date")
            for app in day.get("apps", []):
                for nm in app.get("nm", []):
                    flattened_data.append({
                        "date": date,
                        "nmId": nm.get("nmId"),
                        "name": nm.get("name"),
                        "views": nm.get("views"),
                        "clicks": nm.get("clicks"),
                        "ctr": nm.get("ctr"),
                        "cpc": nm.get("cpc"),
                        "sum": nm.get("sum"),
                        "atbs": nm.get("atbs"),
                        "orders": nm.get("orders"),
                        "cr": nm.get("cr"),
                        "shks": nm.get("shks"),
                        "sum_price": nm.get("sum_price"),
                        "advertId": advertId,
                    })
    
    # Create DataFrame
    df = pd.DataFrame(flattened_data)
    
    # Convert and clean date column
    if not df.empty and "date" in df.columns:
        df["date"] = pd.to_datetime(df["date"]).dt.tz_localize(None)
    
    return df

# Process all datasets with one line each
df_guten = flatten_campaign_data(all_campaign_data_guten)
df_giper = flatten_campaign_data(all_campaign_data_giper)
df_kitchen = flatten_campaign_data(all_campaign_data_kitchen)
df_smart = flatten_campaign_data(all_campaign_data_smart)

df_guten


,date,nmId,name,views,clicks,ctr,cpc,sum,atbs,orders,cr,shks,sum_price,advertId
0,2025-05-14 03:00:00,236559228,Эпилятор женский электрический Silk-epil 1-010,70,1,1.43,10.62,10.62,0,0,0.0,0,0,21204098
1,2025-05-14 03:00:00,236556054,Эпилятор женский электрический Silk-epil 3-031,80,2,2.50,5.92,11.84,0,0,0.0,0,0,21204098
2,2025-05-14 03:00:00,236558116,Эпилятор женский электрический Silk-epil 3-202,67,4,5.97,2.61,10.43,0,0,0.0,0,0,21204098
3,2025-05-14 03:00:00,110511380,Эпилятор женский электрический Silk-epil 3-176,35,0,0.00,0.00,5.22,0,0,0.0,0,0,21204098
4,2025-05-14 03:00:00,236558115,Эпилятор женский электрический Silk-epil 3-000,74,1,1.35,11.36,11.36,0,0,0.0,0,0,21204098
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
902,2025-05-14 03:00:00,264140482,Вытяжка кухонная 60 см встраиваемая телескопич...,0,1,0.00,0.00,0.00,0,0,0.0,0,0,19297685
903,2025-05-14 03:00:00,286300575,Вытяжка кухонная 60 см подвесная ECO,2,0,0.00,0.00,0.27,0,0,0.0,0,0,19297685
904,2025-05-14 03:00:00,252275488,Вытяжка кухонная 60 см встраиваемая телескопич...,81,4,4.94,2.21,8.84,1,0,0.0,0,0,19297685
905,2025-05-14 03:00:00,252275487,Вытяжка кухонная 60 см встраиваемая телескопич...,79,4,5.06,2.03,8.13,0,0,0.0,0,0,19297685


In [12]:
def group_and_aggregate(df, project_name):

    grouped_df = (
        df.groupby([df["date"].dt.date, "nmId","advertId"], as_index=False)
        .agg({
            "date": "first",  # Keep the first date (to retain the day)
            "name": "first",  # Keep the first name (or customize this logic)
            "views": "sum",
            "clicks": "sum",
            "ctr": "mean",  # Sum or average, depending on your needs
            "cpc": "mean",  # Use mean for 'cpc' (cost per click)
            "sum": "sum",
            "atbs": "sum",
            "orders": "sum",
            "cr": "mean",  # Use mean for 'cr' (conversion rate)
            "shks": "sum",
            "sum_price": "sum",
            "advertId": "first"  # Keep the first 'advertId' (no summing)
        })
    )
    
    # Add project and marketplace info
    grouped_df['Project'] = project_name
    grouped_df['Marketplace'] = 'Wildberries'
    
    return grouped_df

# Process each DataFrame using the function
df_grouped_guten = group_and_aggregate(df_guten, 'WB-GutenTech')
df_grouped_giper = group_and_aggregate(df_giper, 'WB-ГиперМаркет')
df_grouped_kitchen = group_and_aggregate(df_kitchen, 'WB-KitchenAid')
df_grouped_smart = group_and_aggregate(df_smart, 'WB-Smart-Market')

# Combine all DataFrames
df_grouped_combined_campaigns = pd.concat([df_grouped_guten, df_grouped_giper, df_grouped_kitchen, df_grouped_smart], ignore_index=True)


# Display the grouped DataFrame
df_grouped_combined_campaigns

,nmId,date,name,views,clicks,ctr,cpc,sum,atbs,orders,cr,shks,sum_price,advertId,Project,Marketplace
0,25088481,2025-05-14 03:00:00,Эпилятор женский электрический Silk-epil 9-880,497,20,3.480000,8.850000,111.56,1,0,0.000000,0,0,21204285,WB-GutenTech,Wildberries
1,25623718,2025-05-14 03:00:00,Бритва мужская электрическая S3 300BT с триммером,276,12,3.840000,2.913333,28.97,4,3,12.500000,3,21513,21202453,WB-GutenTech,Wildberries
2,25623718,2025-05-14 03:00:00,Бритва мужская электрическая S3 300BT с триммером,459,29,6.230000,3.663333,93.86,2,0,0.000000,0,0,21202745,WB-GutenTech,Wildberries
3,25624340,2025-05-14 03:00:00,Электробритва Series 3 300s,763,26,2.436667,1.793333,75.64,3,0,0.000000,0,0,21202453,WB-GutenTech,Wildberries
4,25624340,2025-05-14 03:00:00,Электробритва Series 3 300s,772,53,6.540000,6.606667,157.52,8,1,1.076667,1,6084,21202745,WB-GutenTech,Wildberries
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
569,235597032,2025-05-14 03:00:00,Эпилятор женский электрический Silk-epil 7-030,17,0,0.000000,0.000000,1.16,0,0,0.000000,0,0,20822563,WB-Smart-Market,Wildberries
570,235599972,2025-05-14 03:00:00,Электробритва Series 9 Pro+ 9510s,111,5,5.236667,1.046667,12.24,1,0,0.000000,0,0,15562363,WB-Smart-Market,Wildberries
571,235604574,2025-05-14 03:00:00,Триммер для бороды и усов BT5421,254,4,1.110000,1.186667,30.35,0,0,0.000000,0,0,19597465,WB-Smart-Market,Wildberries
572,235638522,2025-05-14 03:00:00,Эпилятор Silk-epil 5 5-000,4,0,0.000000,0.000000,0.28,0,0,0.000000,0,0,20822563,WB-Smart-Market,Wildberries


In [13]:
# Merge the grouped DataFrame with the filtered_df to add additional columns
df_final = df_grouped_combined_campaigns.merge(
    filtered_df[["advertId", "endTime", "createTime", "startTime","name", "status", "type"]],
    on="advertId",
    how="left"
)

# Drop the columns 'ctr', 'cpc', and 'cr'
df_final = df_final.drop(columns=["ctr", "cpc", "cr"])

# Rename the columns 'name_x' and 'name_y'
df_final.rename(
    columns={
        "name_x": "name_product",  # Rename 'name_x' to 'name_product'
        "name_y": "name_campaign"  # Rename 'name_y' to 'name_campaign'
    },
    inplace=True
)

# Display the final DataFrame
df_final

,nmId,date,name_product,views,clicks,sum,atbs,orders,shks,sum_price,advertId,Project,Marketplace,endTime,createTime,startTime,name_campaign,status,type
0,25088481,2025-05-14 03:00:00,Эпилятор женский электрический Silk-epil 9-880,497,20,111.56,1,0,0,0,21204285,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2024-11-27,BRAUN_9 серия Эпиляторы_07.11.2024,Идут показы,Аукцион
1,25623718,2025-05-14 03:00:00,Бритва мужская электрическая S3 300BT с триммером,276,12,28.97,4,3,3,21513,21202453,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_Электробритвы_07.11.2024_РКП,Идут показы,Автоматическая кампания
2,25623718,2025-05-14 03:00:00,Бритва мужская электрическая S3 300BT с триммером,459,29,93.86,2,0,0,0,21202745,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2025-01-09,BRAUN_3 серия Электробритвы_07.11.2024,Идут показы,Аукцион
3,25624340,2025-05-14 03:00:00,Электробритва Series 3 300s,763,26,75.64,3,0,0,0,21202453,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_Электробритвы_07.11.2024_РКП,Идут показы,Автоматическая кампания
4,25624340,2025-05-14 03:00:00,Электробритва Series 3 300s,772,53,157.52,8,1,1,6084,21202745,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2025-01-09,BRAUN_3 серия Электробритвы_07.11.2024,Идут показы,Аукцион
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
569,235597032,2025-05-14 03:00:00,Эпилятор женский электрический Silk-epil 7-030,17,0,1.16,0,0,0,0,20822563,WB-Smart-Market,Wildberries,2100-01-01,2024-10-22,2025-05-05,Braun_Эпиляторы_22.10.2024,Идут показы,Автоматическая кампания
570,235599972,2025-05-14 03:00:00,Электробритва Series 9 Pro+ 9510s,111,5,12.24,1,0,0,0,15562363,WB-Smart-Market,Wildberries,2100-01-01,2024-03-21,2025-02-20,Braun_Электробритвы_22.10.2024,Идут показы,Автоматическая кампания
571,235604574,2025-05-14 03:00:00,Триммер для бороды и усов BT5421,254,4,30.35,0,0,0,0,19597465,WB-Smart-Market,Wildberries,2100-01-01,2024-08-29,2025-02-20,Braun_Триммеры_22.10.2024,Идут показы,Автоматическая кампания
572,235638522,2025-05-14 03:00:00,Эпилятор Silk-epil 5 5-000,4,0,0.28,0,0,0,0,20822563,WB-Smart-Market,Wildberries,2100-01-01,2024-10-22,2025-05-05,Braun_Эпиляторы_22.10.2024,Идут показы,Автоматическая кампания


## Статистика карточек товаров по дням
https://seller-analytics-api.wildberries.ru/api/v2/nm-report/detail/history

Response Schema: application/json

- data / Array of objects 
    - nmID / integer <int32> / Артикул W
    - imtName / string / Наименование карточки товар
    - vendorCode/ string/ Артикул продавц
    - history/ Array of objects / Array
        - dt/ string <date> / Дата
        - openCardCount / integer <int32> / Количество переходов в карточку товар
        - addToCartCount / integer <int32> / Положили в корзину, шту
        - ordersCount / integer <int32> / Заказали товаров, ш
        - ordersSumRub / integer <int32> / Заказали на сумму, руб/ buyoutsCount/ integer <int32/ Выкупили товаров, шт
        - buyoutsSumRub / integer <int32> / Выкупили на сумму, руб
        - buyoutPercent / integer <int32> / Процент выкупа, % (Какой процент посетителей, заказавших товар, его выкупили. Без учёта товаров, которые еще доставляются покупателю.)
        - addToCartConversion / numbe / Конверсия в корзину, % (Какой процент посетителей, открывших карточку товара, добавили товар в корзину)
        - cartToOrderConversion / integer <int32> / Конверсия в заказ, % (Какой процент посетителей, добавивших товар в корзину, сделали заказ)
- error/ boolean / Флаг ошибки
- errorText/ string / Описание ошибки
- additionalErrors / Array of objects / Дополнительные ошибки
        - field/ string / Структура, где допущена ошибки
        - description/ string / Описание

In [14]:
# API endpoint
url = "https://seller-analytics-api.wildberries.ru/api/v2/nm-report/detail/history"

# Define headers and project names
projects = {
    'WB-GutenTech': headers_guten,
    'WB-ГиперМаркет': headers_giper,
    'WB-KitchenAid': headers_kitchen,
    'WB-Smart-Market': headers_smart
}

# Calculate yesterday's date
#yesterday = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')

# Define the period for the report (only yesterday)
period = {
    "begin": yesterday,
    "end": yesterday
}

# Function to send a single POST request
def send_request(nm_ids, period, headers):
    request_body = {
        "nmIDs": nm_ids,
        "period": period
    }
    response = requests.post(url, headers=headers, json=request_body)
    return response

# Function to fetch data in batches
def fetch_data_in_batches(nm_ids, period, headers, project_name):
    all_data = []
    batch_size = 20  # Maximum allowed nmIDs per request
    requests_per_minute = 3  # Maximum allowed requests per minute
    interval = 60 / requests_per_minute  # Time interval between requests in seconds

    for i in range(0, len(nm_ids), batch_size):
        batch = nm_ids[i:i + batch_size]
        response = send_request(batch, period, headers)

        if response.status_code == 200:
            try:
                data = response.json()
                if not data.get('error') and 'data' in data:
                    all_data.extend(data['data'])
                    print(f"Data retrieved successfully for batch {i // batch_size + 1} for {project_name}")
                else:
                    print(f"Error in response for batch {i // batch_size + 1}: {data.get('errorText', 'No error text')}")
            except ValueError as e:
                print(f"Failed to decode JSON for batch {i // batch_size + 1}: {e}")
        else:
            print(f"Error for batch {i // batch_size + 1}: {response.status_code}, {response.text}")
            break

        time.sleep(interval)  # Respect the rate limit

    return all_data

# List to store DataFrames for each project
dataframes = []

# Process each project
for project_name, headers in projects.items():
    # Filter the DataFrame for the current project
    filtered_df = df_final[df_final['Project'] == project_name]
    unique_nmId_values = filtered_df['nmId'].unique().tolist()
    print(f"Total unique nmId values for {project_name}:", len(unique_nmId_values))  # Debugging line

    # Fetch data for all unique nmId values
    all_data = fetch_data_in_batches(unique_nmId_values, period, headers, project_name)

    # Flatten the nested 'history' data for easier analysis
    flattened_data = []
    for item in all_data:
        nmID = item['nmID']
        imtName = item['imtName']
        vendorCode = item['vendorCode']
        for history in item['history']:
            history_entry = {
                'nmID': nmID,
                'imtName': imtName,
                'vendorCode': vendorCode,
                'dt': history['dt'],
                'openCardCount': history['openCardCount'],
                'addToCartCount': history['addToCartCount'],
                'addToCartConversion': history['addToCartConversion'],
                'ordersCount': history['ordersCount'],
                'ordersSumRub': history['ordersSumRub'],
                'cartToOrderConversion': history['cartToOrderConversion'],
                'buyoutsCount': history['buyoutsCount'],
                'buyoutsSumRub': history['buyoutsSumRub'],
                'buyoutPercent': history['buyoutPercent']
            }
            flattened_data.append(history_entry)

    # Convert the flattened data to a DataFrame
    df_copy = pd.DataFrame(flattened_data)
    print(f"Data for {project_name}:\n", df_copy.head())

    # Add the DataFrame to the list
    dataframes.append(df_copy)

# Concatenate all DataFrames into a single DataFrame
combined_df = pd.concat(dataframes, ignore_index=True)

# Display the combined DataFrame
print("Combined DataFrame:\n", combined_df.head())
df_copy=combined_df.copy()

Total unique nmId values for WB-GutenTech: 263
Data retrieved successfully for batch 1 for WB-GutenTech
Data retrieved successfully for batch 2 for WB-GutenTech
Data retrieved successfully for batch 3 for WB-GutenTech
Data retrieved successfully for batch 4 for WB-GutenTech
Data retrieved successfully for batch 5 for WB-GutenTech
Data retrieved successfully for batch 6 for WB-GutenTech
Data retrieved successfully for batch 7 for WB-GutenTech
Data retrieved successfully for batch 8 for WB-GutenTech
Data retrieved successfully for batch 9 for WB-GutenTech
Data retrieved successfully for batch 10 for WB-GutenTech
Data retrieved successfully for batch 11 for WB-GutenTech
Data retrieved successfully for batch 12 for WB-GutenTech
Data retrieved successfully for batch 13 for WB-GutenTech
Data retrieved successfully for batch 14 for WB-GutenTech
Data for WB-GutenTech:
        nmID                                            imtName  \
0  25088481     Эпилятор женский электрический Silk-epil 9-8

In [15]:
df_copy=combined_df.copy()
df_final_copy=df_final.copy()

In [16]:
df_copy.head()

,nmID,imtName,vendorCode,dt,openCardCount,addToCartCount,addToCartConversion,ordersCount,ordersSumRub,cartToOrderConversion,buyoutsCount,buyoutsSumRub,buyoutPercent
0,25088481,Эпилятор женский электрический Silk-epil 9-880,Б0035166,2025-05-14,64,4,6,0,0,0,0,0,0
1,25623718,Бритва мужская электрическая S3 300BT с триммером,Б0046737,2025-05-14,110,6,5,1,7171,17,0,0,0
2,25624340,Электробритва Series 3 300s,Б0027735,2025-05-14,198,15,8,2,12148,13,0,0,0
3,25628178,Бритва мужская электрическая S3 300s,Б0046736,2025-05-14,111,5,5,0,0,0,0,0,0
4,25993470,Погружной су-вид SV 1200 Pro Smart,CM-00-00001725,2025-05-14,1,0,0,0,0,0,0,0,0


In [17]:
df_final_copy.head()

,nmId,date,name_product,views,clicks,sum,atbs,orders,shks,sum_price,advertId,Project,Marketplace,endTime,createTime,startTime,name_campaign,status,type
0,25088481,2025-05-14 03:00:00,Эпилятор женский электрический Silk-epil 9-880,497,20,111.56,1,0,0,0,21204285,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2024-11-27,BRAUN_9 серия Эпиляторы_07.11.2024,Идут показы,Аукцион
1,25623718,2025-05-14 03:00:00,Бритва мужская электрическая S3 300BT с триммером,276,12,28.97,4,3,3,21513,21202453,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_Электробритвы_07.11.2024_РКП,Идут показы,Автоматическая кампания
2,25623718,2025-05-14 03:00:00,Бритва мужская электрическая S3 300BT с триммером,459,29,93.86,2,0,0,0,21202745,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2025-01-09,BRAUN_3 серия Электробритвы_07.11.2024,Идут показы,Аукцион
3,25624340,2025-05-14 03:00:00,Электробритва Series 3 300s,763,26,75.64,3,0,0,0,21202453,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_Электробритвы_07.11.2024_РКП,Идут показы,Автоматическая кампания
4,25624340,2025-05-14 03:00:00,Электробритва Series 3 300s,772,53,157.52,8,1,1,6084,21202745,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2025-01-09,BRAUN_3 серия Электробритвы_07.11.2024,Идут показы,Аукцион


In [ ]:
# Convert 'day' and 'dt' to datetime for accurate merging
df_final_copy['date'] = pd.to_datetime(df_final_copy['date']).dt.date
df_copy['dt'] = pd.to_datetime(df_copy['dt']).dt.date

# Rename columns in df2 to match df1 for merging
df_copy.rename(columns={'nmID': 'nmId', 'dt': 'day'}, inplace=True)
df_final_copy.rename(columns={'date': 'day'}, inplace=True)

# Merge the DataFrames on 'nmId', 'day', and 'Project'
merged_df_2 = pd.merge(
    df_copy,
    df_final_copy[['nmId', 'day', 'name_product', 'views', 'clicks', 'sum', 'atbs',
       'orders', 'shks', 'sum_price', 'advertId', 'Project', 'Marketplace',
       'endTime', 'createTime', 'startTime', 'name_campaign', 'status',
       'type']],
    on=['nmId', 'day'],
    how='left'
)

# Fill NaN values with 0 for ordersCount and ordersSumRub
merged_df_2['ordersCount'].fillna(0, inplace=True)
merged_df_2['ordersSumRub'].fillna(0, inplace=True)
merged_df_2['addToCartCount'].fillna(0, inplace=True)


# Display the merged DataFrame
merged_df_2

C:\Users\User\AppData\Local\Temp\ipykernel_5588\2208407275.py:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df_2['ordersCount'].fillna(0, inplace=True)
C:\Users\User\AppData\Local\Temp\ipykernel_5588\2208407275.py:22: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exa

,nmId,imtName,vendorCode,day,openCardCount,addToCartCount,addToCartConversion,ordersCount,ordersSumRub,cartToOrderConversion,...,sum_price,advertId,Project,Marketplace,endTime,createTime,startTime,name_campaign,status,type
0,25088481,Эпилятор женский электрический Silk-epil 9-880,Б0035166,2025-05-14,64,4,6,0,0,0,...,0,21204285,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2024-11-27,BRAUN_9 серия Эпиляторы_07.11.2024,Идут показы,Аукцион
1,25623718,Бритва мужская электрическая S3 300BT с триммером,Б0046737,2025-05-14,110,6,5,1,7171,17,...,21513,21202453,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_Электробритвы_07.11.2024_РКП,Идут показы,Автоматическая кампания
2,25623718,Бритва мужская электрическая S3 300BT с триммером,Б0046737,2025-05-14,110,6,5,1,7171,17,...,0,21202745,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2025-01-09,BRAUN_3 серия Электробритвы_07.11.2024,Идут показы,Аукцион
3,25624340,Электробритва Series 3 300s,Б0027735,2025-05-14,198,15,8,2,12148,13,...,0,21202453,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_Электробритвы_07.11.2024_РКП,Идут показы,Автоматическая кампания
4,25624340,Электробритва Series 3 300s,Б0027735,2025-05-14,198,15,8,2,12148,13,...,6084,21202745,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2025-01-09,BRAUN_3 серия Электробритвы_07.11.2024,Идут показы,Аукцион
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
569,235597032,Эпилятор женский электрический Silk-epil 7-030,7500435225120,2025-05-14,11,0,0,0,0,0,...,0,20822563,WB-Smart-Market,Wildberries,2100-01-01,2024-10-22,2025-05-05,Braun_Эпиляторы_22.10.2024,Идут показы,Автоматическая кампания
570,235599972,Электробритва Series 9 Pro+ 9510s,7500435218023,2025-05-14,37,3,8,1,29977,33,...,0,15562363,WB-Smart-Market,Wildberries,2100-01-01,2024-03-21,2025-02-20,Braun_Электробритвы_22.10.2024,Идут показы,Автоматическая кампания
571,235604574,Триммер для бороды и усов BT5421,4210201448051,2025-05-14,58,0,0,0,0,0,...,0,19597465,WB-Smart-Market,Wildberries,2100-01-01,2024-08-29,2025-02-20,Braun_Триммеры_22.10.2024,Идут показы,Автоматическая кампания
572,235638522,Эпилятор Silk-epil 5 5-000,7500435225069,2025-05-14,9,0,0,0,0,0,...,0,20822563,WB-Smart-Market,Wildberries,2100-01-01,2024-10-22,2025-05-05,Braun_Эпиляторы_22.10.2024,Идут показы,Автоматическая кампания


In [19]:
merged_df_2

,nmId,imtName,vendorCode,day,openCardCount,addToCartCount,addToCartConversion,ordersCount,ordersSumRub,cartToOrderConversion,...,sum_price,advertId,Project,Marketplace,endTime,createTime,startTime,name_campaign,status,type
0,25088481,Эпилятор женский электрический Silk-epil 9-880,Б0035166,2025-05-14,64,4,6,0,0,0,...,0,21204285,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2024-11-27,BRAUN_9 серия Эпиляторы_07.11.2024,Идут показы,Аукцион
1,25623718,Бритва мужская электрическая S3 300BT с триммером,Б0046737,2025-05-14,110,6,5,1,7171,17,...,21513,21202453,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_Электробритвы_07.11.2024_РКП,Идут показы,Автоматическая кампания
2,25623718,Бритва мужская электрическая S3 300BT с триммером,Б0046737,2025-05-14,110,6,5,1,7171,17,...,0,21202745,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2025-01-09,BRAUN_3 серия Электробритвы_07.11.2024,Идут показы,Аукцион
3,25624340,Электробритва Series 3 300s,Б0027735,2025-05-14,198,15,8,2,12148,13,...,0,21202453,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_Электробритвы_07.11.2024_РКП,Идут показы,Автоматическая кампания
4,25624340,Электробритва Series 3 300s,Б0027735,2025-05-14,198,15,8,2,12148,13,...,6084,21202745,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2025-01-09,BRAUN_3 серия Электробритвы_07.11.2024,Идут показы,Аукцион
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
569,235597032,Эпилятор женский электрический Silk-epil 7-030,7500435225120,2025-05-14,11,0,0,0,0,0,...,0,20822563,WB-Smart-Market,Wildberries,2100-01-01,2024-10-22,2025-05-05,Braun_Эпиляторы_22.10.2024,Идут показы,Автоматическая кампания
570,235599972,Электробритва Series 9 Pro+ 9510s,7500435218023,2025-05-14,37,3,8,1,29977,33,...,0,15562363,WB-Smart-Market,Wildberries,2100-01-01,2024-03-21,2025-02-20,Braun_Электробритвы_22.10.2024,Идут показы,Автоматическая кампания
571,235604574,Триммер для бороды и усов BT5421,4210201448051,2025-05-14,58,0,0,0,0,0,...,0,19597465,WB-Smart-Market,Wildberries,2100-01-01,2024-08-29,2025-02-20,Braun_Триммеры_22.10.2024,Идут показы,Автоматическая кампания
572,235638522,Эпилятор Silk-epil 5 5-000,7500435225069,2025-05-14,9,0,0,0,0,0,...,0,20822563,WB-Smart-Market,Wildberries,2100-01-01,2024-10-22,2025-05-05,Braun_Эпиляторы_22.10.2024,Идут показы,Автоматическая кампания


## Insert the data

In [20]:
password = os.getenv('ClickHouse')
# Define connection parameters
client = get_client(
    host='rc1a-j5ou9lq30ldal602.mdb.yandexcloud.net',  # Your Yandex Cloud ClickHouse host
    port=8443,                                          # Yandex Cloud uses port 8443 for HTTPS
    username='user1',                           # Your ClickHouse username
    password= password,                           # Your ClickHouse password
    database='user1',                            # Your database name
    secure=True,                                        # Use HTTPS
    verify=False                                        # Disable SSL certificate verification 
    # Define the data to insert
)

In [21]:
# Ensure date columns are in the correct format for ClickHouse
merged_df_2['day'] = pd.to_datetime(merged_df_2['day'])  # Convert to datetime


# Debugging: Check the data types of the DataFrame
print("Data types of merged_df:")
print(merged_df_2.dtypes)

# Ensure the DataFrame has the correct columns
columns = [
    'nmId', 'day', 'name_product', 'views', 'clicks', 'sum', 'atbs', 'orders', 'shks',
    'sum_price', 'advertId', 'Project', 'Marketplace','endTime', 'createTime', 'startTime', 
    'name_campaign', 'status', 'type','ordersCount', 'ordersSumRub','addToCartCount'
]

# Reorder columns to match the expected order
merget_df_copy_2 = merged_df_2[columns]

# Convert DataFrame to a list of tuples for bulk insertion
data = [tuple(row) for row in merget_df_copy_2.to_numpy()]

# Debugging: Check the structure of the data
print("Sample data to insert:", data[:5])  # Print the first 5 rows to check the structure

# Define the table name
table_name = 'campaign_data_wb'

# Use the insert method for bulk insertion
client.insert(table_name, data, column_names=columns)
print("Data inserted successfully!")

Data types of merged_df:
nmId                              int64
imtName                          object
vendorCode                       object
day                      datetime64[ns]
openCardCount                     int64
addToCartCount                    int64
addToCartConversion               int64
ordersCount                       int64
ordersSumRub                      int64
cartToOrderConversion             int64
buyoutsCount                      int64
buyoutsSumRub                     int64
buyoutPercent                     int64
name_product                     object
views                             int64
clicks                            int64
sum                             float64
atbs                              int64
orders                            int64
shks                              int64
sum_price                         int64
advertId                          int64
Project                          object
Marketplace                      object
endTime        